In [10]:
import numpy as np
import pandas as pd
import datetime
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data import TimeFrame, TimeFrameUnit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from zoneinfo import ZoneInfo
from datetime import timedelta
import ta

In [2]:
#Alpaca API key and secret
API_KEY = os.getenv('ALPACA_KEY')
API_SECRET = os.getenv('ALPACA_SECRET')
client = StockHistoricalDataClient(API_KEY, API_SECRET)                       

In [3]:
symbol = "NVDA" 

In [29]:
now = datetime.datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 90),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 1000,                                               # specify limit
)
df = client.get_stock_bars(req).df
df.head()

open     high      low    close  \
symbol timestamp                                                        
NVDA   2025-05-29 14:00:00+00:00  141.5908  141.890  139.290  141.790   
       2025-05-29 15:00:00+00:00  141.7900  141.940  140.330  140.540   
       2025-05-29 16:00:00+00:00  140.5400  141.050  140.055  140.455   
       2025-05-29 17:00:00+00:00  140.4500  140.610  138.320  138.790   
       2025-05-29 18:00:00+00:00  138.7950  139.485  137.910  139.025   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
NVDA   2025-05-29 14:00:00+00:00  70902036.0     586340.0  140.533759  
       2025-05-29 15:00:00+00:00  37512784.0     306845.0  141.305217  
       2025-05-29 16:00:00+00:00  27732114.0     236348.0  140.488018  
       2025-05-29 17:00:00+00:00  33756043.0     265658.0  139.485838  
       2025-05-29 18:00:00+00:00  32781554.0     252031.0  138.857676

In [30]:
df_symbol = df.loc[symbol].copy()

# Moving averages
df_symbol['SMA5'] = df_symbol['close'].rolling(window=5).mean()
df_symbol['SMA10'] = df_symbol['close'].rolling(window=10).mean()
df_symbol['SMA50'] = df_symbol['close'].rolling(window=50).mean()

# Price change %
df_symbol['Price_Change'] = df_symbol['close'].pct_change()

# RSI
df_symbol['RSI'] = ta.momentum.RSIIndicator(df_symbol['close'], window=14).rsi()

# MACD + Signal
macd = ta.trend.MACD(df_symbol['close'])
df_symbol['MACD'] = macd.macd()
df_symbol['MACD_Signal'] = macd.macd_signal()

In [31]:
df_symbol['Target'] = (df_symbol['close'].shift(-1) > df_symbol['close']).astype(int)

In [32]:
fig = go.Figure(data=[
    go.Candlestick(
        x=df_symbol.index,
        open=df_symbol['open'],
        high=df_symbol['high'],
        low=df_symbol['low'],
        close=df_symbol['close'],
        name=symbol
    )
])

# Layout
fig.update_layout(
    title=f"{symbol} - Last 1 Month (Hourly Data)",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()

In [33]:
def create_model(input_shape):
    from tensorflow.keras.regularizers import l2
    model = Sequential([ 
        LSTM(32, input_shape=input_shape, return_sequences=True, recurrent_dropout=0.2, dropout=0.2),
        LSTM(16, input_shape=input_shape, return_sequences=False, recurrent_dropout=0.2, dropout=0.2, kernel_regularizer=l2(0.001)),
        Dense(8, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
        ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [34]:
def prepare_data(df, look_back = 10):
    features =['close', 'SMA5', 'SMA10', 'SMA50', 
               'Price_Change', 'RSI', 'MACD', 'MACD_Signal']
    df_clean = df.dropna()
    
    if df_clean.empty:
        raise ValueError("DataFrame is empty after dropping NaN values.")
    
    if len(df_clean) <= look_back+1:
        raise ValueError(f"Not enough data. Need at least {look_back+1} data points, got {len(df_clean)}.")
    
    scalar = StandardScaler()
    scaled_data = scalar.fit_transform(df_clean[features])
    
    x, y = [], []
    
    for i in range(look_back, len(scaled_data)-1):
        x.append(scaled_data[i-look_back:i])
        y.append(df_clean['Target'].iloc[i])
    
    print(f"Created {len(x)} training sequences.")
    return np.array(x), np.array(y), scalar

In [35]:
look_back = 10
x, y, scaler = prepare_data(df_symbol, look_back=look_back)

print("X shape:", x.shape)
print("Y shape:", y.shape)

Created 928 training sequences.
X shape: (928, 10, 8)
Y shape: (928,)


In [36]:
from sklearn.model_selection import train_test_split

# Split 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=False  # no shuffle since stock data is time series
)

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (742, 10, 8) (742,)
Test shape: (186, 10, 8) (186,)


In [37]:
input_shape = (x_train.shape[1], x_train.shape[2])  # (timesteps, features)
model = create_model(input_shape)

history = model.fit(
    x_train, y_train,
    epochs=100,               # can increase for better learning
    batch_size=32,
    validation_data=(x_test, y_test),
    verbose=1
)


Epoch 1/100


c:\AI_Assisstant\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.4838 - loss: 0.7445 - val_accuracy: 0.5215 - val_loss: 0.7404
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5337 - loss: 0.7350 - val_accuracy: 0.5215 - val_loss: 0.7399
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5337 - loss: 0.7325 - val_accuracy: 0.5215 - val_loss: 0.7363
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5202 - loss: 0.7281 - val_accuracy: 0.5215 - val_loss: 0.7398
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5216 - loss: 0.7242 - val_accuracy: 0.5215 - val_loss: 0.7358
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5472 - loss: 0.7199 - val_accuracy: 0.5215 - val_loss: 0.7319
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5189 - loss: 0.7203 - val_accuracy: 0.5215 - val_loss: 0.7342
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5310 - loss: 0.7201 - val_accuracy: 0.5215 - val_loss: 0.

In [38]:
now = datetime.datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Hour),
    start=now - timedelta(days=30),
    limit=1000,
)
df = client.get_stock_bars(req).df
df_symbol = df.loc[symbol].copy()


In [39]:
df_symbol = df.loc[symbol].copy()

# Moving averages
df_symbol['SMA5'] = df_symbol['close'].rolling(window=5).mean()
df_symbol['SMA10'] = df_symbol['close'].rolling(window=10).mean()
df_symbol['SMA50'] = df_symbol['close'].rolling(window=50).mean()

# Price change %
df_symbol['Price_Change'] = df_symbol['close'].pct_change()

# RSI
df_symbol['RSI'] = ta.momentum.RSIIndicator(df_symbol['close'], window=14).rsi()

# MACD + Signal
macd = ta.trend.MACD(df_symbol['close'])
df_symbol['MACD'] = macd.macd()
df_symbol['MACD_Signal'] = macd.macd_signal()

In [40]:
df_symbol['Target'] = (df_symbol['close'].shift(-1) > df_symbol['close']).astype(int)

In [41]:
# Re-scale data using the same features
x, y, scaler = prepare_data(df_symbol, look_back=10)

# Get the latest sequence (last 10 timesteps)
latest_x = x[-1].reshape(1, x.shape[1], x.shape[2])


Created 291 training sequences.


In [42]:
pred_prob = model.predict(latest_x)[0][0]
prediction = "📈 UP" if pred_prob > 0.5 else "📉 DOWN"

print(f"Model predicts this Friday's stock movement: {prediction} (confidence={pred_prob:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
Model predicts this Friday's stock movement: 📈 UP (confidence=0.58)
